# Umgang mit den Lücken
### Interpolation

In dieser Datei werden die Lücken der Zeitreihen der endgültig ausgewählten Stationen mittels zeitbasierter linearer Interpolation geschlossen. Dazu wird "Datum / Uhrzeit" als datetime definiert, sodass fehlende Zeitpunkte als Zeilen mit NaN-Werten erscheinen. Anschließend werden fehlende Werte innerhalb der Zeitreihe zeitbasiert interpoliert (method="time"). Dabei werden ausschließlich fehlende Werte innerhalb der Datn interpoliert ("limit_area="inside"). Die Ränder der zeitreihen werden nicht gefüllt. Dies ist jedoch unproblematisch, da bereits bei der Auswahl der Stationen festgelegt wurde, dass alle gewählten Zeitreihen den definierten Zeitraum vollständig abdecken und somit an den Rändern gültige Einträge vorliegen.

Interpolierte Werte werden in der zusätzlichen Spalte "Interpoliert" markiert. Das bedeutet, ein Eintrag ist "True", wenn der Wert zuvor gefehlt hat (NaN) und durch Interpolation ergänzt wurde. Werte die nicht interpoliert wurden, erhalten in dieser Spalte "False". Die interpolierten Dateien werden mit der Endung "interpolated.csv" gespeichert, wobei "_filtered_shortened" der Übersichtlichkeit wegen ersetzt wird.

In [4]:
from pathlib import Path
import pandas as pd
import glob

# glob all files in "Auswahl Landespegel Hydrologie" with the ending "_filtered_shortened.csv".
files=glob.glob("Datensätze/UDO-LUBW/Auswahl Landespegel Hydrologie/Gewässer/*/*/*_filtered_shortened.csv")
print(f"Found {len(files)} files")

# interpolate every gap in the respective csv file
def interpolate_gaps():
    interpolated = []
    for p in files:
        p = Path(p)

        # read the respective file as a data frame.
        df = pd.read_csv(p, sep=",", encoding="utf-8")

        # store the data frame as a copy of the data frame to work with it without missunderstandings.
        df = df.copy()
        # define the column "Datum / Uhrzeit" as a real date time.
        df["Datum / Uhrzeit"] = pd.to_datetime(df["Datum / Uhrzeit"], errors="coerce")

        # convert the column "Wert in m^3/s to a real numeric value.
        df["Wert in m^3/s"] = df["Wert in m^3/s"].astype(float)

        # maybe "Datum / Uhrzeit" is by any case not sorted.
        df = df.sort_values("Datum / Uhrzeit")
        
        # set the column "Datum / Uhrzeit" as index and in hourly frequence. Now the gaps are rows with NaNs but the Index is a real date time.
        df = df.set_index("Datum / Uhrzeit").asfreq("h")

        # set marker for the interpolation marker. True if respective df["Wert in m^3"] is nan false if not
        was_nan = df["Wert in m^3/s"].isna()

        # The interpolation of the NaNs in the column "Wert in m^3/s but just inside the data frame.
        df["Wert in m^3/s"] = df["Wert in m^3/s"].interpolate(method="time", limit_area="inside")

        # new column which marks interpolated values
        df["Interpoliert"] = was_nan & df["Wert in m^3/s"].notna()

        # fills the NaNs in "Stationsname" and "Gewässer" with the first entry of the respective column.
        for c in ("Stationsname", "Gewässer"):
            if c in df.columns and df[c].notna().any():
                df[c] = df[c].fillna(df[c].iloc[0])
                
        # reset the index and change the "index" back to "Datum / Uhrzeit" to avoid mistakes for repeated execution. 
        df = df.reset_index().rename(columns={"index": "Datum / Uhrzeit"})

        # create new data frame with the respective columns
        cols = [c for c in ("Stationsname", "Gewässer", "Datum / Uhrzeit", "Wert in m^3/s", "Interpoliert") if c in df.columns]

        # store those as a data frame
        df = df[cols]
        
        # replace old ending with new ending and store it as a csv file
        out_path = p.with_name(p.name.replace("_filtered_shortened", "_interpolated"))
        df.to_csv(out_path, index=False)

        print(f"interpolated: {p.name} to {out_path.name} ({len(df)} rows)")
        interpolated.append(out_path)

    print(f"\n Ready. interpolated {len(interpolated)} of {len(files)} files.")

interpolate_gaps()


        


Found 77 files
interpolated: Oberensingen Aich-Hydrologische Landespegel_filtered_shortened.csv to Oberensingen Aich-Hydrologische Landespegel_interpolated.csv (131496 rows)
interpolated: Lauben Aitrach-Hydrologische Landespegel_filtered_shortened.csv to Lauben Aitrach-Hydrologische Landespegel_interpolated.csv (131496 rows)
interpolated: Ettlingen Alb-Hydrologische Landespegel_filtered_shortened.csv to Ettlingen Alb-Hydrologische Landespegel_interpolated.csv (131496 rows)
interpolated: Riegel-pumpwerksteg-Hydrologische Landespegel_filtered_shortened.csv to Riegel-pumpwerksteg-Hydrologische Landespegel_interpolated.csv (131496 rows)
interpolated: Gießen Argen-Hydrologische Landespegel_filtered_shortened.csv to Gießen Argen-Hydrologische Landespegel_interpolated.csv (131496 rows)
interpolated: Achstetten Baierzer Rot-Hydrologische Landespegel_filtered_shortened.csv to Achstetten Baierzer Rot-Hydrologische Landespegel_interpolated.csv (131496 rows)
interpolated: Blaubeuren Blau-Hydrologi